## Analysis of Data and Classification Models in Letter Recognition
### by Joshua Gabella
_______________________________________________________________________________________________________________________________


Imports

In [35]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import svm

In [36]:
df = pd.read_csv("letter-recognition.data", index_col=False)
data = df.to_numpy()
HKdata = data[np.logical_or(data[:,0]=='H', data[:,0] == 'K'), :]
MYdata = data[np.logical_or(data[:,0]=='M', data[:,0] == 'Y'), :]
DOdata = data[np.logical_or(data[:,0]=='D', data[:,0] == 'O'), :]

### H and K Analysis

In [37]:
training_data, test_data = train_test_split(HKdata, test_size=0.1, random_state=0)
training_y = training_data[:, 0]
training_X = training_data[:, 1:]
test_y = test_data[:, 0]
test_X = test_data[:, 1:]

In [43]:
cs = [0.1, 1, 3, 5, 10]
scores = {param_value : cross_val_score(svm.SVC(kernel='rbf', C = param_value), training_X, training_y, cv=5) for param_value in cs}


[0.95849057 0.9509434  0.98113208 0.95471698 0.95849057]
{0.1: array([0.92075472, 0.90188679, 0.92830189, 0.92075472, 0.93962264]), 1: array([0.95849057, 0.9509434 , 0.98113208, 0.95471698, 0.95849057]), 3: array([0.96603774, 0.96981132, 0.98490566, 0.96226415, 0.96603774]), 5: array([0.97358491, 0.97358491, 0.98490566, 0.96226415, 0.97735849]), 10: array([0.98113208, 0.96981132, 0.97735849, 0.96603774, 0.97735849])}
